In [1]:
import requests
import csv
import time

BASE_URL = "https://vinfastauto.com/vn_vi/get-locator"
OUTPUT_CSV = "vinfast_stations_by_id.csv"

def _get(d, *keys, default=None):
    """Lấy giá trị lồng nhau an toàn: _get(obj, 'a','b','c')."""
    cur = d
    for k in keys:
        if not isinstance(cur, dict):
            return default
        cur = cur.get(k)
        if cur is None:
            return default
    return cur

def _ensure_list(x):
    if x is None:
        return []
    if isinstance(x, list):
        return x
    return [x]

def _parse_connectors(nested, payload):
    """
    Trả về:
      - connectors: list tên loại cổng (ví dụ: ['CCS2', 'Type2-AC', 'CHAdeMO'])
      - connector_count: tổng số cổng (nếu API có field count thì dùng, không thì đếm theo danh sách)
    Hỗ trợ cả key 'connector' lẫn 'connecter', và cả cấu trúc có 'ports'/'sockets'.
    """
    # Tìm mảng mô tả cổng
    candidates = []
    for root in (nested, payload):
        # thường thấy
        cand = root.get('connector') or root.get('connecter') or root.get('connectors')
        if cand:
            candidates = _ensure_list(cand)
            break
        # một số API gom trong extra_data
        extra = root.get('extra_data') or {}
        cand = extra.get('connector') or extra.get('connecter') or extra.get('connectors')
        if cand:
            candidates = _ensure_list(cand)
            break

    connectors = []
    connector_count = 0

    # Nếu phần tử là dict, cố gắng đọc tên/loại và số lượng
    for item in _ensure_list(candidates):
        if isinstance(item, dict):
            # Tên loại cổng có thể nằm trong nhiều key khác nhau
            name = (item.get('name')
                    or item.get('type')
                    or item.get('standard')
                    or item.get('connector')
                    or item.get('code'))
            if name:
                connectors.append(str(name))
            # Số lượng cổng cho từng loại (nếu có)
            qty = (item.get('count')
                   or item.get('quantity')
                   or item.get('num_ports')
                   or item.get('ports'))
            if isinstance(qty, int):
                connector_count += qty
            elif isinstance(qty, list):
                connector_count += len(qty)
        else:
            # Phần tử là string -> chỉ là tên loại
            connectors.append(str(item))

    # Một số API không để trong 'connector' mà để trong 'ports'/'sockets'
    if not connectors:
        for root in (nested, payload, _get(nested, 'extra_data', default={}), _get(payload, 'extra_data', default={})):
            for key in ('ports', 'sockets'):
                arr = root.get(key)
                if isinstance(arr, list):
                    for p in arr:
                        if isinstance(p, dict):
                            nm = p.get('type') or p.get('name') or p.get('standard')
                            if nm:
                                connectors.append(str(nm))
                    connector_count = connector_count or len(arr)

    # Nếu vẫn chưa có tổng, lấy theo độ dài danh sách (best-effort)
    if connector_count == 0:
        connector_count = len(connectors)

    # Loại bỏ trùng lặp, giữ thứ tự
    seen = set()
    dedup = []
    for c in connectors:
        if c not in seen and c is not None:
            seen.add(c)
            dedup.append(c)

    return dedup, connector_count

def fetch_station(station_id: int):
    url = f"{BASE_URL}/{station_id}"
    try:
        r = requests.get(url, timeout=7)
        if r.status_code != 200:
            return None

        root = r.json() or {}
        payload = root.get("data", {}) or {}
        nested = payload.get("data", {}) or {}

        # Tên trạm (depot_name)
        depot_name = (
            _get(nested, "extra_data", "depot_name")
            or nested.get("depot_name")
            or payload.get("depot_name")
            or _get(payload, "extra_data", "depot_name")
            or nested.get("name")
            or payload.get("name")
        )

        # Loại trạm: car_charging_station / bike_charging_station
        category = (
            nested.get("category")
            or payload.get("category")
            or _get(nested, "extra_data", "category")
            or _get(payload, "extra_data", "category")
        )

        # Địa chỉ
        address = (
            nested.get("address")
            or payload.get("address")
            or _get(nested, "extra_data", "address")
            or _get(payload, "extra_data", "address")
        )

        # Tọa độ
        coords = nested.get("coordinates") or {}
        lat = coords.get("latitude") or payload.get("lat") or _get(nested, "latitude")
        lng = coords.get("longitude") or payload.get("lng") or _get(nested, "longitude")

        # Trạng thái depot
        depot_status = _get(nested, "extra_data", "depot_status") or _get(payload, "extra_data", "depot_status")

        # Admin IDs
        district_id = payload.get("district_id")
        province_id = payload.get("province_id")

        # Connectors & tổng số cổng
        connectors, connector_count = _parse_connectors(nested, payload)

        return {
            "id": station_id,
            "depot_name": depot_name,
            "category": category,
            "depot_status": depot_status,
            "address": address,
            "latitude": lat,
            "longitude": lng,
            "district_id": district_id,
            "province_id": province_id,
            "connector_count": connector_count,
            "connectors": "|".join(connectors) if connectors else None,  # nối bằng dấu |
        }
    except (requests.RequestException, ValueError):
        return None

def main():
    results = []
    # Bạn có thể điều chỉnh dải ID theo nhu cầu
    for sid in range(14000, 30001):
        info = fetch_station(sid)
        if info:
            results.append(info)
        # Delay nhỏ để tránh rate limit
        time.sleep(0.05)

    # Xuất CSV
    fieldnames = [
        "id",
        "depot_name",
        "category",
        "depot_status",
        "address",
        "latitude",
        "longitude",
        "district_id",
        "province_id",
        "connector_count",
        "connectors",
    ]
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    print(f"Đã thu về {len(results)} trạm hợp lệ, lưu vào {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Đã thu về 14355 trạm hợp lệ, lưu vào vinfast_stations_by_id.csv
